In [3]:
# Install latest OpenAI SDK (already >=1.0)
!pip install --upgrade openai -q

# Load API key securely
from google.colab import userdata
import openai
import re

client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# --- Data Layer ---
sample_dataset = {
    "What are the top 5 customers by revenue?": "SELECT customer_id, SUM(revenue) FROM sales GROUP BY customer_id ORDER BY SUM(revenue) DESC LIMIT 5;",
    "How many orders were placed in January?": "SELECT COUNT(*) FROM orders WHERE order_date BETWEEN '2023-01-01' AND '2023-01-31';",
    "List all products out of stock": "SELECT product_name FROM products WHERE stock = 0;"
}

# --- Governance Layer ---
def violates_policy(user_input: str) -> str:
    blocked_phrases = [
        r"buy|sell.*(stock|crypto|bitcoin|investment)",
        r"should I invest|what should I invest",
        r"social security number|SSN|passport",
        r"give.*advice"
    ]
    for pattern in blocked_phrases:
        if re.search(pattern, user_input, re.IGNORECASE):
            return "Your query violates usage policy (e.g., investment or personal data)."
    return ""

# --- Intelligence Layer (Modern SDK) ---
def generate_sql(user_input):
    system_prompt = (
        "You are an assistant that translates English questions into SQL. "
        "Only output valid SQL queries without any explanation."
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.2,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# --- User Experience Layer ---
def chat_interface():
    print("SQL Translator Bot")
    print("Type your question or 'exit' to stop.\n")
    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit"]:
            print("Session ended.")
            break

        # Governance check
        violation = violates_policy(user_input)
        if violation:
            print(violation)
            continue

        # Generate SQL
        sql = generate_sql(user_input)

        print("\nTranslated SQL:")
        print(sql)

        # Feedback
        correct = sample_dataset.get(user_input)
        if correct:
            if sql.strip(";") == correct.strip(";"):
                print("This matches the known correct SQL.")
            else:
                print("Note: Output differs from expected SQL.")
        else:
            print("No reference available for this question.\n")

# Run it
chat_interface()


SQL Translator Bot
Type your question or 'exit' to stop.

You: top customers

Translated SQL:
SELECT * 
FROM Customers 
ORDER BY Total_Purchases DESC 
LIMIT 10;
No reference available for this question.

You: out of stock

Translated SQL:
Your request is not clear. Please provide more details such as the table name and the column name that contains the stock information. However, assuming you have a table named 'products' and a column named 'stock', the SQL query would be:

```sql
SELECT * FROM products WHERE stock = 0;
```
No reference available for this question.



KeyboardInterrupt: Interrupted by user